In [2]:
import numpy as np
import csv
!pip install tensorflow
import tensorflow as tf
!pip install chess
import chess
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Conv2D, Input

# Function to encode chess board from FEN string
# Function to encode the board from FEN
def encode_board_from_fen(fen):
    """Convert FEN string to a tensor representation suitable for neural network input"""
    board = chess.Board(fen)

    # Initialize 8x8x13 tensor (12 channels for pieces + 1 channel for promotion potential)
    tensor = np.zeros((8, 8, 13), dtype=np.float32)

    # Map from piece to channel index
    piece_to_channel = {
        'P': 0, 'N': 1, 'B': 2, 'R': 3, 'Q': 4, 'K': 5,  # White pieces
        'p': 6, 'n': 7, 'b': 8, 'r': 9, 'q': 10, 'k': 11  # Black pieces
    }

    # Fill the tensor with piece positions
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece:
            rank, file = chess.square_rank(square), chess.square_file(square)
            channel = piece_to_channel[piece.symbol()]
            tensor[7-rank, file, channel] = 1  # 7-rank to flip board orientation

    # Add promotion potential channel - mark pawns that are one move away from promotion
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece:
            rank, file = chess.square_rank(square), chess.square_file(square)
            if piece.piece_type == chess.PAWN:
                # White pawns on 7th rank or black pawns on 2nd rank
                if (piece.color == chess.WHITE and rank == 6) or \
                   (piece.color == chess.BLACK and rank == 1):
                    tensor[7-rank, file, 12] = 1  # Promotion potential

    # Add turn, castling rights and en passant state
    # These would be additional channels but omitted for brevity

    return tensor

# Function to create a move mask from legal moves
# Improved function to convert a move to an index in the output vector
def move_to_index(move):
    """Convert a chess move to a unique index"""
    from_square = move.from_square
    to_square = move.to_square

    # Regular moves
    if not move.promotion:
        return from_square * 64 + to_square

    # Promotion moves
    # Use a more reliable encoding for promotions
    # Base (4096) + source square (0-63) * 32 + destination file (0-7) * 4 + promotion type (0-3)
    to_file = chess.square_file(to_square)

    promo_type = {
        chess.KNIGHT: 0,
        chess.BISHOP: 1,
        chess.ROOK: 2,
        chess.QUEEN: 3
    }[move.promotion]

    return 4096 + from_square * 32 + to_file * 4 + promo_type

# Function to convert an index back to a chess move
def index_to_move(index, board):
    """Convert an index back to a chess move"""
    if index < 4096:  # Regular move
        from_square = index // 64
        to_square = index % 64
        return chess.Move(from_square, to_square)
    else:  # Promotion move
        index -= 4096
        from_square = index // 32
        remaining = index % 32
        to_file = remaining // 4
        promo_type = remaining % 4

        # Check if there's a piece at from_square that can promote
        piece = board.piece_at(from_square)
        if not piece or piece.piece_type != chess.PAWN:
            # Cannot promote if there's no pawn at from_square
            return None

        # Determine promotion rank based on piece color
        to_rank = 7 if piece.color == chess.WHITE else 0
        to_square = chess.square(to_file, to_rank)

        # Create promotion move
        promotion_piece = [chess.KNIGHT, chess.BISHOP, chess.ROOK, chess.QUEEN][promo_type]
        return chess.Move(from_square, to_square, promotion=promotion_piece)

# Function to create a move mask from legal moves
def create_move_mask(board):
    """Create a binary mask of legal moves"""
    legal_moves = list(board.legal_moves)

    # Size matches 4096 regular moves + space for promotions
    # 4096 + 64*8*4 = 4096 + 2048 = 6144 should be enough
    mask = np.zeros(6144, dtype=np.float32)

    # Map each legal move to an index in the output vector
    for move in legal_moves:
        move_index = move_to_index(move)
        if move_index < len(mask):
            mask[move_index] = 1
        else:
            print(f"Warning: Move {move} mapped to index {move_index} outside mask bounds of {len(mask)}")

    return mask

# Create the neural network model with matching output size
def create_model():
    """Create a neural network model for chess move prediction"""
    model = Sequential([
        Input(shape=(8, 8, 13)),  # Updated to include promotion potential channel
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        GlobalAveragePooling2D(),
        Dense(1024, activation='relu'),
        Dense(6144, activation='sigmoid')  # Match the mask size of 6144
    ])

    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

def generate_training_data(num_positions=1000):
    """Generate training data from random positions"""
    X = []
    y = []
    dataset_x=[]
    dataset_y=[]
    dataset_z=[]
    for _ in range(num_positions):
        # Create a board with some moves played
        board = chess.Board()
        num_moves = np.random.randint(0, 50)

        for _ in range(num_moves):
            if board.is_game_over():
                break

            legal_moves = list(board.legal_moves)
            if not legal_moves:
                break
            for moves in legal_moves:
              if(len(moves.uci())==5):
                  X.append(encode_board_from_fen(board.fen()))
                  y.append(create_move_mask(board))
                  dataset_x.append(board.fen())
                  dataset_y.append(list(board.legal_moves))
                  dataset_z.append(len(list(board.legal_moves)))
                  break



            move = np.random.choice(legal_moves)
            board.push(move)

        # Encode the board
        dataset_x.append(board.fen())
        X.append(encode_board_from_fen(board.fen()))
        # Create the target mask of legal moves
        y.append(create_move_mask(board))
        dataset_y.append(list(board.legal_moves))
        dataset_z.append(len(list(board.legal_moves)))

# Assuming dataset_x and dataset_y are already populated

    with open('chess_data.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['FEN', 'LegalMoves','NumberOfLegalMoves'])
        for fen, legal_moves,num_moves in zip(dataset_x, dataset_y,dataset_z):
            legal_moves_str = ','.join(str(move.uci()) for move in legal_moves)
            writer.writerow([fen, legal_moves_str,num_moves])


    return np.array(X), np.array(y)

def predict_legal_moves(model, fen, top_n=None):
    """Predict legal moves from a FEN string

    Args:
        model: The trained neural network model
        fen: FEN string representing the board position
        top_n: Number of top moves to return (None for all legal moves)
    """
    board = chess.Board(fen)
    board_tensor = encode_board_from_fen(fen)
    board_tensor = np.expand_dims(board_tensor, axis=0)  # Add batch dimension

    predictions = model(board_tensor, training=False).numpy()[0]


    # Create a mask of actual legal moves
    legal_moves_mask = create_move_mask(board)

    # Apply mask and get moves
    legal_predictions = predictions * legal_moves_mask

    # Get all indices where legal moves exist
    move_indices = np.where(legal_moves_mask > 0)[0]

    # Create list of (move, probability) pairs
    moves_with_probs = []
    for idx in move_indices:
        move = index_to_move(idx, board)
        moves_with_probs.append(move)
        # moves_with_probs.append((move, legal_predictions[idx]))

    # Sort by probability in descending order
    # moves_with_probs.sort(key=lambda x: x[1], reverse=True)

    # Return all or top N moves
    if top_n is not None:
        return moves_with_probs[:top_n]
    return moves_with_probs

# Main function to demonstrate usage

    # Create and train the model
model = create_model()
X, y = generate_training_data(5000)

model.fit(X, y, epochs=50, batch_size=32, validation_split=0.1)

model.save('my_chess_model.keras')  # Recommended Keras format




Epoch 1/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 25s 145ms/step - accuracy: 1.4607e-04 - loss: 0.1656 - val_accuracy: 0.0000e+00 - val_loss: 0.0204
Epoch 2/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 17s 112ms/step - accuracy: 0.0000e+00 - loss: 0.0203 - val_accuracy: 0.0000e+00 - val_loss: 0.0202
Epoch 3/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 8s 52ms/step - accuracy: 0.0077 - loss: 0.0202 - val_accuracy: 0.0154 - val_loss: 0.0200
Epoch 4/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 8s 51ms/step - accuracy: 5.0496e-04 - loss: 0.0198 - val_accuracy: 0.0000e+00 - val_loss: 0.0198
Epoch 5/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 8s 51ms/step - accuracy: 0.0017 - loss: 0.0198 - val_accuracy: 0.0308 - val_loss: 0.0198
Epoch 6/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 7s 50ms/step - accuracy: 0.0066 - loss: 0.0199 - val_accuracy: 0.0058 - val_loss: 0.0198
Epoch 7/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 7s 48ms/step - accuracy: 0.0036 - loss: 0.0199 - val_accuracy: 0.0019 - val_loss: 0.0197
Epoch 8/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 7s 49ms/step - accuracy: 0.01